# 1-Webcrawller Script
Date created: 13-04-2023  
Created by: Jayden Dzierbicki  
Last updated: 14-04-2023  

The purpose of this notebook is to collect data on XRP, a cryptocurrency, from two different sources: historic price data and forum posts. The notebook scrapes data from the following websites:

- https://finance.yahoo.com/quote/XRP-AUD/history?p=XRP-AUD (for XRP price in AUD)
- https://www.cryptocompare.com/coins/xrp/forum (for XRP forum posts)
- https://www.investing.com/crypto/xrp/chat (for XRP forum posts)

All collected data is stored in a mySQL database. A CSV backup of the data is also created and saved to the task1 folder with this notebook.

It's important to note that the mySQL database schema includes tables for both price data and forum posts, with column names and data types specified for each. While this script only collects data from two sources, it demonstrates how web scraping can be used to collect data from multiple sources and store it in a database for analysis.

One potential issue with this data collection process is that there may be biases in the forum posts that are collected, as they are only from two websites. Additionally, there may be technical challenges in scraping the data from these websites. By acknowledging these limitations, readers can better interpret the data and its potential biases.

In [1]:
#Requirments to work
import time # Provides time-related functions
from datetime import datetime, timedelta, date # Provides date and time-related functions
from bs4 import BeautifulSoup # For parsing HTML and XML documents
import pandas as pd # For data manipulation and analysis
import dateparser # For parsing date strings
from selenium import webdriver # For automated web testing
from selenium.webdriver.common.keys import Keys # For simulating keyboard keys
from selenium.webdriver.chrome.options import Options # For setting up Chrome options
from tqdm import tqdm # For displaying progress bars
import getpass # For getting a password without displaying it on the screen
from sqlalchemy import create_engine # For connecting to SQL databases
from yahoo_fin.stock_info import get_data # For getting historical stock data from Yahoo Finance


# Function to load data into mySQL
def load_data_mySQL(database_name, table_name, df):
    user = 'root'

    # Prompt the user for a password
    password = getpass.getpass("Enter your MySQL password: ")
    host = 'localhost'
    port = 3306
    database = database_name
    engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")

    # Write the DataFrame to a SQL table
    table_name =  table_name
    df.to_sql(table_name, engine, if_exists='replace', index=False)

    # Close the connection
    engine.dispose()

## cryptocompare.com crawler
Will need to undergo a simple ETL proccess
- Step 1: Extract data from cryptocompare.com
- Step 2: Transform data in prepration for initial storage (add ID, convert datetime to date and time)
- Step 3: Load data into mySQL

In [6]:
# Extract Data

class cryptocompareCrawler():
    def __init__(self, start_link):
        self.link_to_explore = start_link
        self.comments = pd.DataFrame(columns=['date', 'comment'])
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        self.driver = webdriver.Chrome(executable_path='C:/webdrivers/chromedriver', options=chrome_options)
        self.pagecount = 1
        self.next = True

    def run(self):
        try:
            post_data = self.scrape_crypto_forum(self.link_to_explore)
            self.extract_data(post_data)
            self.save_data_to_file()
        except:
            print("Cannot get the page " + self.link_to_explore)
            raise
            

    def scrape_crypto_forum(self, url, scroll_duration=timedelta(hours=1)):
        self.driver.get(url)

        scroll_interval = 6
        total_iterations = int(scroll_duration.total_seconds() // scroll_interval)

        progress_bar = tqdm(range(total_iterations),  dynamic_ncols=True, desc="Scraping posts")

        start_time = datetime.now()
        post_data = []
        prev_post_count = 0
        iteration = 0
        while iteration < total_iterations:
            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(scroll_interval)

            html_content = self.driver.page_source
            soup = BeautifulSoup(html_content, "html.parser")
            posts = soup.find_all("div", {"class": "post-content"})

            if len(posts) > prev_post_count:
                for post in posts[prev_post_count:]:
                    comment_element = post.find("div", {"class": "content-body"})
                    date_element = post.find("div", {"class": "item-ago ng-binding"})
                    if comment_element and date_element:
                        date_str = date_element['title']
                        parsed_date = dateparser.parse(date_str, settings={"RELATIVE_BASE": datetime.now()})
                        comment = comment_element.text.replace('\n', '').strip()
                        post_data.append({"comment": comment, "date": parsed_date})

                prev_post_count = len(posts)

            iteration += 1
            progress_bar.update(1)

        progress_bar.close()
        return post_data

    def extract_data(self, post_data):
        for data in post_data:
            comment_str = data['comment']
            standardized_date = data['date'].strftime('%Y-%m-%d %H:%M:%S')
            self.comments.loc[len(self.comments)] = [standardized_date, comment_str]

    def save_data_to_file(self):
        today_date = datetime.today().strftime('%Y-%m-%d')
        self.comments.to_csv(f'cryptocompare_{today_date}.csv', index=False)

    def close_spider(self):
        self.driver


In [ ]:
# Extract data
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--no-sandbox')

# Initialize the class with the base URL
crawler = cryptocompareCrawler('https://www.cryptocompare.com/coins/xrp/forum')

# Run the crawler
crawler.run()

# Close the spider
crawler.close_spider()

C:\Users\jayde\AppData\Local\Temp/ipykernel_20188/553188809.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(executable_path='C:/webdrivers/chromedriver', options=chrome_options)


Scraping posts:   0%|                                                                          | 0/600 [00:00<?, ?it/s]

Scraping posts:   0%|                                                                | 1/600 [00:06<1:04:53,  6.50s/it]

Scraping posts:   0%|▏                                                               | 2/600 [00:13<1:05:14,  6.55s/it]

Scraping posts:   0%|▎                                                               | 3/600 [00:19<1:05:52,  6.62s/it]

Scraping posts:   1%|▍                                                               | 4/600 [00:26<1:06:42,  6.72s/it]

Scraping posts:   1%|▌                                                               | 5/600 [00:33<1:07:50,  6.84s/it]

Scraping posts

Scraping posts:  10%|██████▌                                                        | 62/600 [15:04<3:36:04, 24.10s/it]

Scraping posts:  10%|██████▌                                                        | 63/600 [15:31<3:43:09, 24.93s/it]

Scraping posts:  11%|██████▋                                                        | 64/600 [15:57<3:46:05, 25.31s/it]

Scraping posts:  11%|██████▊                                                        | 65/600 [16:23<3:47:17, 25.49s/it]

Scraping posts:  11%|██████▉                                                        | 66/600 [16:49<3:48:38, 25.69s/it]

Scraping posts:  11%|███████                                                        | 67/600 [17:18<3:55:53, 26.55s/it]

Scraping posts:  11%|███████▏                                                       | 68/600 [17:45<3:57:42, 26.81s/it]

Scraping posts:  12%|███████▏                                                       | 69/600 [18:14<4:01:56, 27.34s/it]

Scraping posts:  12%|███████▎   

Scraping posts:  21%|████████████▌                                               | 126/600 [1:04:17<9:06:21, 69.16s/it]

Scraping posts:  21%|████████████▋                                               | 127/600 [1:05:23<8:57:45, 68.21s/it]

Scraping posts:  21%|████████████▊                                               | 128/600 [1:06:37<9:11:24, 70.10s/it]

Scraping posts:  22%|████████████▉                                               | 129/600 [1:07:48<9:13:13, 70.47s/it]

Scraping posts:  22%|█████████████                                               | 130/600 [1:09:07<9:30:53, 72.88s/it]

Scraping posts:  22%|█████████████                                               | 131/600 [1:10:21<9:31:55, 73.17s/it]

Scraping posts:  22%|█████████████▏                                              | 132/600 [1:11:33<9:27:50, 72.80s/it]

Scraping posts:  22%|█████████████▎                                              | 133/600 [1:12:45<9:25:41, 72.68s/it]

Scraping posts:  22%|███████████

Scraping posts:  32%|██████████████████▎                                       | 190/600 [2:57:29<16:32:46, 145.29s/it]

Scraping posts:  32%|██████████████████▍                                       | 191/600 [3:00:10<17:02:26, 149.99s/it]

Scraping posts:  32%|██████████████████▌                                       | 192/600 [3:02:38<16:55:30, 149.34s/it]

Scraping posts:  32%|██████████████████▋                                       | 193/600 [3:05:07<16:51:29, 149.11s/it]

Scraping posts:  32%|██████████████████▊                                       | 194/600 [3:07:57<17:32:56, 155.61s/it]

Scraping posts:  32%|██████████████████▊                                       | 195/600 [3:10:43<17:51:15, 158.71s/it]

Scraping posts:  33%|██████████████████▉                                       | 196/600 [3:13:32<18:08:37, 161.68s/it]

Scraping posts:  33%|███████████████████                                       | 197/600 [3:16:26<18:30:48, 165.38s/it]

Scraping posts:  33%|███████████

Scraping posts:  42%|████████████████████████▌                                 | 254/600 [6:41:54<25:04:05, 260.82s/it]

Scraping posts:  42%|████████████████████████▋                                 | 255/600 [6:46:30<25:26:04, 265.40s/it]

Scraping posts:  43%|████████████████████████▋                                 | 256/600 [6:50:51<25:13:38, 264.01s/it]

Scraping posts:  43%|████████████████████████▊                                 | 257/600 [6:55:17<25:12:45, 264.62s/it]

Scraping posts:  43%|████████████████████████▉                                 | 258/600 [7:00:13<26:02:09, 274.06s/it]

Scraping posts:  43%|█████████████████████████                                 | 259/600 [7:04:36<25:39:53, 270.95s/it]

Scraping posts:  43%|█████████████████████████▏                                | 260/600 [7:09:12<25:43:23, 272.36s/it]

Scraping posts:  44%|█████████████████████████▏                                | 261/600 [7:14:23<26:44:54, 284.05s/it]

Scraping posts:  44%|███████████

In [21]:
# Trasnform Data 

# Add an ID column to the DataFrame
df = pd.read_csv(f'cryptocompare_2023-04-15.csv')
df['id'] = df.index + 1
df['source'] = 'https://www.cryptocompare.com/coins/xrp/forum'
df[['date', 'time']] = df['date'].str.split(' ', n=1, expand=True)
df

,date,comment,id,source,time
0,2023-04-14,"moon soon, coon. mark my words (lol idk - no f...",1,https://www.cryptocompare.com/coins/xrp/forum,08:41:00
1,2023-04-14,Boring xrp at the Moment,2,https://www.cryptocompare.com/coins/xrp/forum,02:27:00
2,2023-04-14,So bankrupted FTX….they recover $7.3 billion i...,3,https://www.cryptocompare.com/coins/xrp/forum,01:20:00
3,2023-04-13,Is UMU what many thought XRP was going to be u...,4,https://www.cryptocompare.com/coins/xrp/forum,19:11:00
4,2023-04-13,Coinbase just deposited Flare into my account....,5,https://www.cryptocompare.com/coins/xrp/forum,00:49:00
...,...,...,...,...,...
6397,2021-02-02,I somewhat feel sorry for the newcommers. I gu...,6398,https://www.cryptocompare.com/coins/xrp/forum,02:30:00
6398,2021-02-02,Veyor..MGI,6399,https://www.cryptocompare.com/coins/xrp/forum,02:28:00
6399,2021-02-02,big jump in ODL volume since the sec lawsuitis...,6400,https://www.cryptocompare.com/coins/xrp/forum,02:27:00
6400,2021-02-02,Funny how humans brain works. In fomo Times ev...,6401,https://www.cryptocompare.com/coins/xrp/forum,02:11:00


In [22]:
# Load data
load_data_mySQL('MA5851_A3', 'cryptocompare_xrp', df)

Enter your MySQL password: ········


## Investing.com crawler
Will need to undergo a simple ETL proccess
- Step 1: Extract data from Investing.com
- Step 2: Transform data in prepration for initial storage (add ID, convert datetime to date and time)
- Step 3: Load data into mySQL

In [6]:
# Code to extract data
# Scrape https://www.investing.com/crypto/xrp/chat 
# Need to crawl multiple pages over 1200 pages! with data from 2019

class investingCrawler():
    def __init__(self, start_link):
        self.link_to_explore = start_link
        self.comments = pd.DataFrame(columns = ['date','comment'])
        self.driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)           
        self.pagecount = 1
        self.next = True
        
    def run(self):
        with tqdm(total=1222, dynamic_ncols=True, desc="Scraping posts") as pbar:
            while self.next:
                if self.pagecount >= 1222:
                    self.save_data_to_file()
                    self.next = False
                try:
                    self.driver.get(self.link_to_explore + "/" + str(self.pagecount))
                    self.driver.implicitly_wait(15)
                    self.extract_data()      
                    self.pagecount = self.pagecount + 1
                    pbar.update(1)
                except:
                    print ("Cannot get the page " + self.link_to_explore)
                    self.next = False
                    raise

    def extract_data(self):
        html_content = self.driver.page_source
        soup = BeautifulSoup(html_content, "html.parser")

        comment_wrappers = soup.find_all("div", {"class": "commentInnerWrapper"})

        count = 0
        for wrapper in comment_wrappers:
            date_element = wrapper.find("span", {"class": "js-date"})
            date_str = date_element["comment-date-formatted"]

            # Parse date string and convert it to a standardized format
            parsed_date = dateparser.parse(date_str)
            standardized_date = parsed_date.strftime('%Y-%m-%d %H:%M:%S')

            comment_element = wrapper.find("span", {"class": "js-text"})
            comment_str = comment_element.text.strip()

            # Adding date and comment to the dataframe
            self.comments.loc[len(self.comments)] = [standardized_date, comment_str]
            count += 1

        return count



    def save_data_to_file(self):
    #we save the dataframe content to a CSV file
        today_date = date.today()
        self.comments.to_csv(f'investing_{today_date}.csv', index=False)
    def close_spider(self):
    #end the session
        self.driver.quit()

In [7]:
# Extract data
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--no-sandbox')

# Initialize the class with the base URL
crawler = investingCrawler('https://www.investing.com/crypto/xrp/chat')

# Run the crawler
crawler.run()

# Close the spider
crawler.close_spider()

C:\Users\jayde\AppData\Local\Temp/ipykernel_27308/3816390378.py:9: DeprecationWarning: use options instead of chrome_options
  self.driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
Scraping posts: 100%|██████████████████████████████████████████████████████████████| 1222/1222 [41:11<00:00,  2.02s/it]


In [8]:
# Transform data
df = pd.read_csv(f'investing_2023-04-14.csv')

# Split the date and time values into separate columns
df[['date', 'time']] = df['date'].str.split(' ', n=1, expand=True)

# Add an ID column & source URL to the DataFrame
df['id'] = df.index + 1
df['source'] = 'https://www.investing.com/crypto/xrp/chat'

# Print to confrim looks correct
df

,date,comment,time,id,source
0,2023-04-14,Still a bargain. Regards.,12:15:57,1,https://www.investing.com/crypto/xrp/chat
1,2023-04-12,btc going higher so will this,23:10:00,2,https://www.investing.com/crypto/xrp/chat
2,2023-04-12,Going back down to .40. No settlement. Market ...,14:39:00,3,https://www.investing.com/crypto/xrp/chat
3,2023-04-12,$1,11:01:00,4,https://www.investing.com/crypto/xrp/chat
4,2023-04-12,Back to sick,07:57:00,5,https://www.investing.com/crypto/xrp/chat
...,...,...,...,...,...
59743,2018-03-18,Same happened in 2013 ....many sold and few go...,03:29:00,59744,https://www.investing.com/crypto/xrp/chat
59744,2018-03-17,Sell it off before it’s too late,15:22:00,59745,https://www.investing.com/crypto/xrp/chat
59745,2018-03-17,No more up for crypto,15:22:00,59746,https://www.investing.com/crypto/xrp/chat
59746,2018-03-16,Now cardano it's going to ********up anytime!,18:43:00,59747,https://www.investing.com/crypto/xrp/chat


In [9]:
# Load data to mySQL ready for task 2
load_data_mySQL('MA5851_A3', 'investingcom_xrp', df)

Enter your MySQL password: ········


## Yahoofinance.com price extraction
Yahoo provide API and tools to obtain data which is much more efficent then scrapping via a built in function for python
Will need to undergo a simple ETL proccess
- Step 1: Extract data from yahoofinance.com
- Step 2: Transform data in prepration for initial storage (add ID, convert datetime to date and time)
- Step 3: Load data into mySQL

In [7]:
# Extract data

df= get_data("XRP-AUD", start_date="01/01/2017", end_date="15/04/2023", index_as_date = False, interval="1d")
today_date = date.today()
df.to_csv(f'xrp_price_yahoo_{today_date}.csv', index=False) #backup

In [4]:
# Transform data
# Add an ID column to the DataFrame
df['id'] = df.index + 1
df

,date,open,high,low,close,adjclose,volume,ticker,id
0,2017-11-10,0.284150,0.285022,0.267963,0.269525,0.269525,184092145,XRP-AUD,1
1,2017-11-11,0.268827,0.279932,0.268188,0.274677,0.274677,175568474,XRP-AUD,2
2,2017-11-12,0.274395,0.274395,0.255044,0.258094,0.258094,328505111,XRP-AUD,3
3,2017-11-13,0.258268,0.267999,0.258315,0.267195,0.267195,174109535,XRP-AUD,4
4,2017-11-14,0.267506,0.280253,0.267401,0.275036,0.275036,166943243,XRP-AUD,5
...,...,...,...,...,...,...,...,...,...
1977,2023-04-10,0.757934,0.763103,0.750791,0.758037,0.758037,825097347,XRP-AUD,1978
1978,2023-04-11,0.757976,0.782039,0.752795,0.779685,0.779685,1375478642,XRP-AUD,1979
1979,2023-04-12,0.779669,0.788820,0.772128,0.777849,0.777849,1813207929,XRP-AUD,1980
1980,2023-04-13,0.777921,0.778733,0.749323,0.755734,0.755734,1587786706,XRP-AUD,1981


In [5]:
# Load data
load_data_mySQL('MA5851_A3', 'xrp_price_yahoo', df)

Enter your MySQL password: ········
